### Pipeline to run research to collect candidates

In [1]:
import sys
sys.path.append('../')

import logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

from components.agents.research_agents import run_mixed_research_agents
from components.prompts.research_agents import FOCUS_AREAS

In [2]:
FOCUS_AREAS.keys()

dict_keys(['reasoning_and_planning', 'agents_and_finance', 'agent_infrastructure', 'retrieval_and_embeddings', 'multimodal_and_generation'])

In [3]:
print(FOCUS_AREAS['reasoning_and_planning'])

Reasoning LLMs, chain-of-thought, inference-time compute, self-reflection, planning with LLMs, MCTS (Monte Carlo Tree Search) for language models, test-time scaling, hallucination reduction and detection, grounding, factuality


### Test run on dict

In [ ]:
research_results = await run_mixed_research_agents(
    {"reasoning_agent": "reasoning agents, chain of thought planning with LLMs, test time compute"}
)

2025-12-24 16:10:34,006 - components.agents.research_agents - INFO - Running 2 tasks
Research agents (mixed):   0%|          | 0/2 [00:00<?, ?it/s]2025-12-24 16:10:34,020 - components.agents.research_agents - INFO - Running OpenAI research agent for focus area: reasoning_agent
2025-12-24 16:10:34,256 - components.agents.research_agents - INFO - Running Anthropic research agent for focus area: reasoning_agent
